In [1]:
import uproot
import numpy as np
import matplotlib.pyplot as plt
import vector  ## added by mds
import awkward as ak
import iminuit
from iminuit import Minuit
from fit_classes import *
from utils import *
import os


In [2]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')


In [7]:

def load_data(decay_type=None, base_path=None, help=False, list_decay_types=False):
    if help:
        print("Usage: load_data(decay_type=None, base_path=None, help=False, list_decay_types=False)")
        print("decay_type: The specific decay type to load, or None to load all decay types.")
        print("base_path: The base path where the data files are located.")
        print("help: Set to True to print this help message.")
        print("list_decay_types: Set to True to list available decay types in the specified location.")
        return

    if base_path is None:
        raise ValueError("Please provide a base path for the data files.")

    all_files = [os.path.join(base_path, file) for file in os.listdir(base_path) if file.endswith(".root")]
    

    if list_decay_types:
        decay_types_per_file = {}
        for filename in all_files:
            file = uproot.open(filename)
            decay_types = set()
            for branch in file.keys():
                if 'DecayTree;1' in branch:
                    decay_type = branch.replace('/DecayTree;1', '')
                    decay_types.add(decay_type)
            decay_types_per_file[filename] = decay_types

        # Check if all files have the same decay types
        all_values = decay_types_per_file.values()
        if all(value == list(all_values)[0] for value in all_values):
            print('All files have the same decay types.')
            print('Available decay types:')
            for decay in list(all_values)[0]:
                print(decay)
        else:
            print('Not all files have the same decay types.')
            for file, decay_types in decay_types_per_file.items():
                print(f'File {file} has the decay types:')
                for decay_type in decay_types:
                    print(decay_type)
        return

    data = []
    for file_path in all_files:
        with uproot.open(file_path) as root_file:
            for key in root_file.keys():
                tree_name = key.split(';')[0]
                if decay_type is None or tree_name == decay_type:
                    data.append(f"{file_path}:{tree_name}/DecayTree")

    return data



In [4]:

def load_mc_data(base_path, selected_mc_types=None, decay_tree=None, list_decay_types=False):
    all_files = [os.path.join(base_path, file) for file in os.listdir(base_path) if file.endswith(".root")]

    if list_decay_types:
        # Extract unique MC types from the file names
        mc_types = set([file.rsplit('_', 1)[0] for file in os.listdir(base_path) if file.endswith('.root')])

        # Display the list of unique MC types to the user
        print("Available MC types:")
        for idx, mc_type in enumerate(mc_types, 1):
            print(f"{idx}. {mc_type}")

        # List available decay types
        decay_types_per_file = {}
        for filename in all_files:
            file = uproot.open(filename)
            decay_types = set()
            for branch in file.keys():
                if 'DecayTree;1' in branch:
                    decay_type = branch.replace('/DecayTree;1', '')
                    decay_types.add(decay_type)
            decay_types_per_file[filename] = decay_types

        # Check if all files have the same decay types
        all_values = decay_types_per_file.values()
        if all(value == list(all_values)[0] for value in all_values):
            print('All files have the same decay types.')
            print('Available decay types:')
            for decay in list(all_values)[0]:
                print(decay)
        else:
            print('Not all files have the same decay types.')
            for file, decay_types in decay_types_per_file.items():
                print(f'File {file} has the decay types:')
                for decay_type in decay_types:
                    print(decay_type)
        return

    if selected_mc_types is not None:
        all_files = [os.path.join(base_path, f"{selected_mc_type}_{year}.root") for selected_mc_type in selected_mc_types for year in ['15MU', '16MU', '17MU', '18MU', '15MD', '16MD', '17MD', '18MD']]

    data = []
    for file_path in all_files:
        with uproot.open(file_path) as root_file:
            for key in root_file.keys():
                tree_name = key.split(';')[0]
                if decay_tree is None or decay_tree in tree_name:
                    data.append(f"{file_path}:{tree_name}/DecayTree")

    return data



In [8]:
# Get the list of files
rd_data_files = load_data(base_path='/share/lazy/Bu2LambdaPHH/NTuples/RD/', decay_type ='B2L0pbarKpKp')

selected_mc_types = ['MCB2L0barPKpKm']
mc_data = load_mc_data(base_path='/share/lazy/Bu2LambdaPHH/NTuples/MC', selected_mc_types=selected_mc_types, decay_tree='B2L0pbarKpKp')

In [9]:
print(f"rd_data_files: {rd_data_files}")
print(f"mc_data: {mc_data}")

rd_data_files: ['/share/lazy/Bu2LambdaPHH/NTuples/RD/L0phh_15MD.root:B2L0pbarKpKp/DecayTree', '/share/lazy/Bu2LambdaPHH/NTuples/RD/L0phh_15MU.root:B2L0pbarKpKp/DecayTree', '/share/lazy/Bu2LambdaPHH/NTuples/RD/L0phh_16MD.root:B2L0pbarKpKp/DecayTree', '/share/lazy/Bu2LambdaPHH/NTuples/RD/L0phh_16MU.root:B2L0pbarKpKp/DecayTree', '/share/lazy/Bu2LambdaPHH/NTuples/RD/L0phh_17MD.root:B2L0pbarKpKp/DecayTree', '/share/lazy/Bu2LambdaPHH/NTuples/RD/L0phh_17MU.root:B2L0pbarKpKp/DecayTree', '/share/lazy/Bu2LambdaPHH/NTuples/RD/L0phh_18MD.root:B2L0pbarKpKp/DecayTree', '/share/lazy/Bu2LambdaPHH/NTuples/RD/L0phh_18MU.root:B2L0pbarKpKp/DecayTree']
mc_data: ['/share/lazy/Bu2LambdaPHH/NTuples/MC/MCB2L0barPKpKm_15MU.root:B2L0pbarKpKp/DecayTree', '/share/lazy/Bu2LambdaPHH/NTuples/MC/MCB2L0barPKpKm_15MU.root:B2L0pbarKpKp/DecayTree/DecayTree', '/share/lazy/Bu2LambdaPHH/NTuples/MC/MCB2L0barPKpKm_16MU.root:B2L0pbarKpKp/DecayTree', '/share/lazy/Bu2LambdaPHH/NTuples/MC/MCB2L0barPKpKm_16MU.root:B2L0pbarKpKp/Deca

In [120]:
arrs_rd = uproot.concatenate(rd_data_files)


KeyboardInterrupt: 

In [160]:
load_mc_data(list_decay_types=True, base_path='/share/lazy/Bu2LambdaPHH/NTuples/MC/')


Available MC types:
1. MCBd2L0barPDm,KPiPi
2. MCBs2L0barPDsm,KKPi
3. MCBd2LcPiPiP,L0barPi
4. MCB2L0barPKpKm
5. MCBu2LcbarPPi,L0barPim
6. MCBd2L0barPDm,KKPi
7. MCBu2L0barPD0bar,KPi
8. MCB2L0PbarKpPip
9. MCBu2L0barLambda1520K,PK
10. MCB2L0PbarKpKp
11. MCBu2L0barPPhi,KK
12. MCB2PPbarPPbarPip
13. MCBu2chic1K
14. MCBu2etacK,PL0barK
15. MCBu2chic2K
16. MCBu2LcbarPPi,Sigma0barPiPi0
17. MCB2L0barPKmPip
18. MCBs2L0barPDssm,Dsgamma,KKPi
19. MCBu2L0barPetaP
20. MCBu2chic0K
21. MCBu2L0barPD0bar,KPiPi0
22. MCB2L0barPJpsi,PPbar
23. MCBu2LcbarPPi,Sigma0barPi
24. MCB2L0barPPipPip
25. MCBu2Psi2SK
26. MCB2L0barPPipPim
27. MCB2L0barPPbarP
28. MCB2L0barPKpPim
29. MCBd2LcbarL0K,PKPi
30. MCBu2JpsiK,PL0barK
31. MCBu2LcbarPPi,L0barPiPi0
32. MCBd2L0barPDsm,KKPi
33. MCBu2L0barPeta,PiPiX
Not all files have the same decay types.
File /share/lazy/Bu2LambdaPHH/NTuples/MC/MCB2L0barPJpsi,PPbar_15MD.root has the decay types:
B2L0barpKpKm
B2L0barppbarp
File /share/lazy/Bu2LambdaPHH/NTuples/MC/MCB2L0barPJpsi,PPbar_15MU.